Define the neural net as a `class` and a method `forward(input)` that returns the output.

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule: `weight = weight - learning_rate * gradient`

### CNN for Classification

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module): # inherit the nn.Module class
    
    def __init__(self):
        super(Net, self).__init__() # to use the (layer class) functions from the parent class
        
        # Define the network architecture 
        # Just the layers !!!
        # NOT how the input gets transformed into the output, that is described in the forward method
        # Like Sequential API in keras
        # Stack layers one on top of the other 
        
        # single channel input, 6 filters each of size 5x5
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
        
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(in_features=16 * 4 * 4, out_features=120) 
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10) # 10 digit classification 
        
    def forward(self, x):
        # Output of conv1 layer is sent to activation function
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(input=x, kernel_size=(2,2))
        
        # Operations afer 2nd conv
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(input=x, kernel_size=(2,2))
        
        # Flatten (by reshaping)
        # -1 indicates torch to infer the other dimension
        x = x.view(-1, self.num_flat_features(x)) # this function is defined below
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # No activation in final layer, i.e., just logits
        
        return x
        
    def num_flat_features(self, x):
        # to get flattened dim, take product all dimensions except the batch dimension
        size = x.size()[1:] 
        num_features = 1
        for s in size:
            num_features *= s
        return num_features 
        

In [3]:
net = Net()
print(net) # Summary of layers

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


- we just have to define the `forward` function
- the `backward` function (where gradients are computed) is automatically defined for you using `autograd`. You can use __any of the Tensor operations in the forward function__ when using the nn Module (without the need to explicitly mention `requires_grad`).

To see the list of learnable parameters, 

In [4]:
params = list(net.parameters())
print(len(params))
print(params[-1]) # final layer weights (randomly initialized)

10
Parameter containing:
tensor([-0.0594, -0.0053, -0.0245,  0.0676, -0.0549,  0.0081, -0.0739, -0.0548,
         0.0424, -0.0926], requires_grad=True)


In [5]:
print(params[0].size())  # conv1's .weight, ie, 6 filters of size 5x5

torch.Size([6, 1, 5, 5])


In [6]:
inp = torch.randn(1, 1, 28, 28)
out = net(inp)
print(out)

tensor([[ 0.0139,  0.0244, -0.0532,  0.0404, -0.0990,  0.0367, -0.1207, -0.0957,
          0.0036, -0.0849]], grad_fn=<ThAddmmBackward>)


Every time a variable is back propagated through, the gradient will be __accumulated instead of being replaced.__ <br>
So when starting out, we clear the buffers and start from scratch by:

In [7]:
net.zero_grad()

In [8]:
# We havent actually loaded the train x and y
# And hence, do not have the loss calculated

# So lets try backproping with RANDOM gradients
out.backward(torch.randn(1, 10))

### Loss Function

In [9]:
out = net(inp)
target = torch.randn(10) # Dummy target
target = target.view(1, -1) # make it [1, 10]

criterion = nn.MSELoss()

loss = criterion(out, target)
print(loss) # Prints out the loss between the output and the dummy target from above

tensor(0.9638, grad_fn=<MseLossBackward>)


So, when we call `loss.backward()`, the whole graph is differentiated w.r.t. the loss, and all Tensors in the graph that has `requires_grad=True` will have their `.grad` Tensor accumulated with the gradient.

Let us look a few steps backward

In [10]:
print(loss.grad_fn)  # MSELoss

In [11]:
print(loss.grad_fn.next_functions[0][0])  # Linear

In [12]:
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

### Backpropagation
To backpropagate the error all we have to do is to `loss.backward()`. You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

In [13]:
# Has ACCUMULATED gradients from out.backward(torch.randn(1, 10)) ~!~!~!
print(net.conv1.bias.grad) 

tensor([-0.0995,  0.0710, -0.0139, -0.0405,  0.0028, -0.0852])


In [14]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

# Calculate gradient of a conv bias term wrt the above calculated loss
print('conv1.bias.grad before backward propagation')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after 1 step of backward prop gradient computation')
print(net.conv1.bias.grad)

conv1.bias.grad before backward propagation
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after 1 step of backward prop gradient computation
tensor([ 0.0013, -0.0170, -0.0096,  0.0104,  0.0117,  0.0125])


### Weight Update Process
- Specify optimizer and learning rate

In [15]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(params = net.parameters(), lr=0.01) # specify the parameters to be optimized

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
out = net(inp)

loss = criterion(out, target)

In [16]:
loss.backward() # compute gradients

In [17]:
optimizer.step()    # Do the update: weight = weight - learning_rate * gradient